# Numerical Benchmarks

In [3]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from qiskit.quantum_info import random_unitary
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
def generate_spam_data(spam_target, noise = 0):
    n = int(np.log2(spam_target.d))
    
    inputs_spam, _ = generate_pauliInput_circuits(n)

    U_prep = inputs_spam
    N_spam = U_prep.shape[0]

    state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, U_prep)
    targets_spam = measurement(state, povm = spam_target.povm)
    targets_spam = targets_spam + tf.cast(tf.random.normal(targets_spam.shape, 0, noise), dtype = precision)
    
    return inputs_spam, targets_spam


def generate_map_data(kraus_target, noise = 0):
    n = int(np.log2(kraus_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None, 
                                            N = None, 
                                            trace=False)
    U_prep, U_basis = inputs_map
    N_map = U_prep.shape[0]
    
    state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = kraus_target.apply_map(state)
    targets_map = measurement(state, U_basis, spam_target.povm)
    targets_map = targets_map + tf.cast(tf.random.normal(targets_map.shape, 0, noise), dtype = precision)
    
    return inputs_map, targets_map

## Retrieve SPAM and Map, No Noise

In [3]:
n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.9

### Retrieve Map without SPAM

In [4]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []
for i in tqdm(range(100)):
    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    kraus_target_list.append(kraus_target)
    #inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)
    
    
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = None,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
pickle.dump(channel_fidelity_list, open("../../data/spam_fidelity_noSpam_noNoise.p", "wb"))

In [2]:
channel_fidelity_list = pickle.load(open("../../data/spam_fidelity_noSpam_noNoise.p", "rb"))
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9738505722699933 0.0007709473079391712


### Retrieve SPAM and Map

In [6]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []

for i in range(100):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    kraus_target_list.append(kraus_target)
    
    inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)

    #train spam model
    spam_model = SPAM(d=d,
    #                  use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
pickle.dump(channel_fidelity_list, open("../../data/spam_fidelity_POVM_noNoise.p", "wb"))

In [3]:
channel_fidelity_list = pickle.load(open("../../data/spam_fidelity_POVM_noNoise.p", "rb"))
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9953371456348762 0.00042363199117524024


### Retrieve SPAM (Corruption Matrix) and Map

In [8]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []
for i in range(100):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    kraus_target_list.append(kraus_target)
    
    inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)

    #train spam model
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
pickle.dump(channel_fidelity_list, open("../../data/spam_fidelity_CM_noNoise.p", "wb"))

In [4]:
channel_fidelity_list = pickle.load(open("../../data/spam_fidelity_CM_noNoise.p", "rb"))
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9948335539875042 0.0004893392324422893


## Retrieve SPAM and Map, with Noise

In [10]:
n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.9

### Retrieve Map without SPAM

In [11]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []
for i in range(100):
    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    kraus_target_list.append(kraus_target)
    #inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target, noise = 0.02)
    
    
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = None,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
pickle.dump(channel_fidelity_list, open("../../data/spam_fidelity_noSpam_noise.p", "wb"))

In [5]:
channel_fidelity_list = pickle.load(open("../../data/spam_fidelity_noSpam_noise.p", "rb"))
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9582258951021818 0.0010699533662570772


### Retrieve SPAM and Map

In [13]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []

for i in range(100):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    kraus_target_list.append(kraus_target)
    
    inputs_spam, targets_spam = generate_spam_data(spam_target, noise = 0.02)
    inputs_map, targets_map = generate_map_data(kraus_target, noise = 0.02)

    #train spam model
    spam_model = SPAM(d=d,
    #                  use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
pickle.dump(channel_fidelity_list, open("../../data/spam_fidelity_POVM_noise.p", "wb"))

In [6]:
channel_fidelity_list = pickle.load(open("../../data/spam_fidelity_POVM_noise.p", "rb"))
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9411520189560173 0.0032781457177947803


### Retrieve SPAM (Corruption Matrix) and Map

In [15]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []

for i in range(100):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    kraus_target_list.append(kraus_target)
    
    inputs_spam, targets_spam = generate_spam_data(spam_target, noise = 0.02)
    inputs_map, targets_map = generate_map_data(kraus_target, noise = 0.02)

    #train spam model
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [16]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
pickle.dump(channel_fidelity_list, open("../../data/spam_fidelity_CM_noise.p", "wb"))

In [7]:
channel_fidelity_list = pickle.load(open("../../data/spam_fidelity_CM_noise.p", "rb"))
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9819299540228137 0.0006322155179331074


## Retrieveing Map with Extreme POVM Errors, No Noise

In [3]:
n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.6

### Full POVM

In [4]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []

for i in range(3):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    kraus_target_list.append(kraus_target)
    
    inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)

    #train spam model
    spam_model = SPAM(d=d,
    #                  use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9919473992330988 0.003102192890534414


### Corruption Matrix

In [8]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
kraus_target_list = []

for i in range(3):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    kraus_target_list.append(kraus_target)
    
    inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)

    #train spam model
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                verbose = False,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
channel_fidelity_list = np.array([channel_fidelity(model.q_map, kraus_target).numpy() for model, kraus_target in zip(model_list, kraus_target_list)])
print(np.mean(channel_fidelity_list), np.std(channel_fidelity_list)/np.sqrt(len(channel_fidelity_list)))

0.9422928366036901 0.012215054611356848


## Invariance of SPAM loss

In [4]:
def apply(state, U):
    return U@state@U.T.conj()

In [36]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 2
d = 2**n
c = 0.9

#spam_model = SPAM(d=d)
#init = (c*init_ideal(d) + (1-c)*spam_model.init).numpy()
#povm = (c*povm_ideal(d)[0] + (1-c)*spam_model.povm[0]).numpy()

#spam_model1 = SPAM(d=2)
#pam_model2 = SPAM(d=2)
#init = kron(spam_model1.init, spam_model2.init).numpy()
#povm = kron(spam_model1.povm[0], spam_model2.povm[0]).numpy()


init = np.array([[0.5, 0, 0, 0], [0, 0.4, 0, 0], [0, 0, 0.1, 0], [0, 0, 0, 0]])
povm = np.array([[1, 0, 0, 0], [0, 0.3, 0, 0], [0, 0, 0.1, 0], [0, 0, 0, 0.2]])
#U_scramble = kron(*[Operator(random_unitary(2)).data for j in range(n)]).numpy()
#U_scramble = Operator(random_unitary(d)).data

#init = apply(init, U_scramble)
#povm = apply(povm, U_scramble)

In [37]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

U_perm = np.array([[0,0,0,1], [0,0,1,0], [0,1,0,0], [1,0,0,0]])
#U_perm = U_scramble@U_perm@U_scramble.T.conj()
#U_perm = np.array([[1,0,0,0], [0,0,1,0], [0,1,0,0], [0,0,0,1]])
#U_perm = kron(*[Operator(random_unitary(2)).data for j in range(n)]).numpy()

for i in range(6**n):
    config = numberToBase(i,6,n)
    #U_prep = prepare_input(config, return_mode = "unitary")
    U_prep = kron(*[Operator(random_unitary(2)).data for j in range(n)]).numpy()
    #U_prep = Operator(random_unitary(d)).data
    
    init_s = apply(init, U_prep)
    EV = np.trace(init_s@povm)
    
    init_perm_s = apply(init, U_prep@U_perm)
    povm_perm = apply(povm, U_perm)
    EV_perm = np.trace(init_perm_s@povm_perm)
    
    print(f"{EV:.3f}", f"{EV_perm:.3f}")

0.348+0.000j 0.348+0.000j
0.480-0.000j 0.480+0.000j
0.408+0.000j 0.408+0.000j
0.475-0.000j 0.475-0.000j
0.264+0.000j 0.264+0.000j
0.501+0.000j 0.501+0.000j
0.349-0.000j 0.349-0.000j
0.612+0.000j 0.612+0.000j
0.399+0.000j 0.399+0.000j
0.440-0.000j 0.440-0.000j
0.280+0.000j 0.280+0.000j
0.526-0.000j 0.526-0.000j
0.350-0.000j 0.350-0.000j
0.204+0.000j 0.204-0.000j
0.297-0.000j 0.297-0.000j
0.221-0.000j 0.221-0.000j
0.607-0.000j 0.607-0.000j
0.470-0.000j 0.470+0.000j
0.259-0.000j 0.259-0.000j
0.245-0.000j 0.245-0.000j
0.285-0.000j 0.285-0.000j
0.226-0.000j 0.226-0.000j
0.402-0.000j 0.402-0.000j
0.472-0.000j 0.472-0.000j
0.458+0.000j 0.458+0.000j
0.287+0.000j 0.287+0.000j
0.311-0.000j 0.311-0.000j
0.271-0.000j 0.271-0.000j
0.292+0.000j 0.292+0.000j
0.274+0.000j 0.274+0.000j
0.588+0.000j 0.588+0.000j
0.333-0.000j 0.333-0.000j
0.308-0.000j 0.308+0.000j
0.252-0.000j 0.252-0.000j
0.403-0.000j 0.403+0.000j
0.249+0.000j 0.249+0.000j
